# Aprendizado de Máquina

### Este documento apenas tem objetivo para fins exploratórios. Ele não agrega a nada sobre o relatório principal. Nele, testei e explorei possibilidades acerca de utilizar algoritmos envolvendo Gradient Boosting, porém, devido a inconscitência dos dados, isto é, ser um Random Walk extremamente volátil, junto a um desbalanceamento de emissões de casos, tornou-se dificultoso um trabalho para se chegar em um modelo que apresentasse uma acurácia consistente, se baseando no erro percentual absoluto médio simétrico.

In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import gzip
import plotly.graph_objects as go

In [3]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import smape_loss

In [1]:
from numpy import ravel
import sklearn.model_selection
from sktime.forecasting.compose import make_reduction
from skopt import dummy_minimize
from lightgbm import LGBMRegressor
from statsmodels.tsa.filters.hp_filter import hpfilter
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import ForecastingGridSearchCV

In [4]:
from sktime.forecasting.naive import NaiveForecaster
from sklearn.model_selection import train_test_split

In [13]:
from lazypredict.Supervised import LazyRegressor

C:\Users\heylu\miniconda3\envs\lazypredict\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
url = 'https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true'
r = requests.get(url, allow_redirects=True)
open('data.csv.gz','wb').write(r.content)
gz = gzip.open('data.csv.gz')
df = pd.read_csv(gz)

## Validação de Modelos

In [7]:
total_de_casos_amazonas = df.query("state == 'AM'").groupby('date').sum()
total_de_casos_amazonas.index = pd.to_datetime(total_de_casos_amazonas.index)
total_de_casos_amazonas.index.freq = 'D'

In [8]:
def to_zero(x):
    if x < 0:
        return 0
    return x

In [157]:
total_de_casos_amazonas.drop(columns=['epi_week', 'ibgeID', 'cod_RegiaoDeSaude'], inplace=True)

KeyError: "['epi_week' 'ibgeID' 'cod_RegiaoDeSaude'] not found in axis"

In [158]:
total_de_casos_amazonas['newDeaths'] = total_de_casos_amazonas['newDeaths'].apply(to_zero)

In [159]:
total_de_casos_amazonas['newCases'] = total_de_casos_amazonas['newCases'].apply(to_zero)

In [161]:
SEED = 4

In [232]:
y = total_de_casos_amazonas['newCases']
_, y = hpfilter(y)
y = y.apply(to_zero).round()
y = y.apply(lambda x: int(x))
X = y.index
X = np.array([int(x) for x in range(len(X))])
y = y.values

In [233]:
split = 7

In [234]:
X_train, X_test = X[:-(split)], X[-(split):]
y_train, y_test = y[:-(split)], y[-(split):]

In [235]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

In [236]:
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

 31%|█████████████████████████▍                                                        | 13/42 [00:00<00:00, 41.73it/s]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range for family GammaDistribution


 81%|██████████████████████████████████████████████████████████████████▍               | 34/42 [00:01<00:00, 22.45it/s]

OrthogonalMatchingPursuitCV model failed to execute
Found array with 1 feature(s) (shape=(393, 1)) while a minimum of 2 is required by OrthogonalMatchingPursuitCV.


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 30.87it/s]


In [237]:
predictions

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
DecisionTreeRegressor,-5.26,-4.21,43.22,0.01
ExtraTreeRegressor,-5.26,-4.21,43.22,0.01
ExtraTreesRegressor,-5.26,-4.21,43.22,0.10
XGBRegressor,-5.42,-4.35,43.77,0.03
BaggingRegressor,-6.94,-5.61,48.69,0.02
RandomForestRegressor,-7.18,-5.82,49.43,0.13
GradientBoostingRegressor,-7.80,-6.33,51.26,0.03
SVR,-7.80,-6.33,51.27,0.01
KNeighborsRegressor,-11.80,-9.67,61.83,0.01
